In [3]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\LEO\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
from tkinter import *
import json
import math
root = Tk()
from tkinter.messagebox import showinfo
from tkinter.filedialog import askopenfilename, asksaveasfilename
import os



f1 = Frame(root, bg="grey", borderwidth=6, relief=SUNKEN)
f1.pack(side=LEFT, fill="y")
l = Label(f1, text="k181182 Faraz Sarwar")
l.pack( pady=142)

f2 = Frame(root, borderwidth=8, bg="grey", relief=SUNKEN)
f2.pack(side=TOP, fill="x")
l = Label(f2, text="VECTOR SPACE MODEL", font="Helvetica 16 bold", fg="red", pady=22)
l.pack()


#=======================================================================


#=======================================================================

    
#------------------------------------------------------------------------------------
#                           CosineSim
#-------------------------------------------------------------------------------------

#=========================MAGNITUDE===========================
def magnitude(values) :
  mag = 0
  for i in range(len(values)) :
    mag = mag + math.pow(values[i], 2)
  
  mag = math.sqrt(mag)
  return mag


#=========================DOT_PRODUCT=========================
def DotProduct(doc,queryy) :
  dotprod = 0
  for i in range (len(doc)) :
    dotprod = dotprod + doc[i]*queryy[i]

  return dotprod

#-------------------------------------------------------------------------------------------




#==========================SAVE_FUNCTION_DISLK============================
def save(invertedIndex,name) :
  import json 
  with open('{0}.json'.format(name), 'w') as ij:
    json.dump(invertedIndex,ij)

  ij.close();
  

#-------------------------------------------------------------------------------
#                                  Query
#-------------------------------------------------------------------------------


#==================================TERM_FREQUENCEY_QUERY=========================
def calTermFrequency_query(queryy,uniquelist) :
  n = 50

  query_tf =  {}

  #InvertedIndex
  for Uword in uniquelist :
    counter = 0 
    counter = queryy.count(Uword)
    if counter >= 1 :
      print("yes i am here ")
    #print("here==",Uword,counter,i)
    if query_tf.get('query') == None :
      query_tf.update({('query'):[]})
    query_tf['query'].append(counter)

  return query_tf

#=================================PRE_PROCESSING_QUERY========================
def PreprocessingQuery(queryy) :
  queryy = stopWordsRemoval_query(queryy)
  queryy = lemmatization_query(queryy)
  #queryy = stemming_query(queryy)
  return queryy

#=============================LEMMATIZATION_QUERY===============================
def lemmatization_query(query) :
  wordnet_lemmatizer = WordNetLemmatizer()

  temp = []

  for word in query :
    newword = wordnet_lemmatizer.lemmatize(''.join(map(str, word)))
    temp.append(newword)

  return temp 

#=============================STOP_WORDS_REMOVAL_QUERY==========================
def stopWordsRemoval_query(queryy) :
    myfile = open('Stopword-List.txt',encoding='utf-8')
    string = myfile.read()
    
    #temp variable to store new words
    temp = []
    for word in queryy :
      if word not in string.split() :
        temp.append(word)
    return temp 



#=========Convert==============
#string to list converter
def Convert(string):
    li = list(string.split(" "))
    return li

#-----------------------------------------------------------------------
                          #RESULT WINDOW FUNCTION
#-----------------------------------------------------------------------
 #---------------ANOTHER WINDOW FOR DOCUMENTS VIEW----------------
def openFile(TextArea,value):
    global file
    print("Here i am ",value)
    file = "E:/Semester_6/IR/Assignment#02/ShortStories/{0}.txt".format(value)
    print(file)
    if file == "":
        file = None
    else:
        #root.title(os.path.basename(file) + " - Notepad")
        TextArea.delete(1.0, END)
        f = open(file, "r",encoding="utf-8")
        TextArea.insert(1.0, f.read())
        f.close()


#================================================
def Result_screen(value,DOC_RES_root) :

  #Add TextArea
  TextArea = Text(DOC_RES_root, font="lucida 13")
  file = None
  TextArea.pack(expand=True, fill=BOTH)

  # Lets create a menubar
  MenuBar = Menu(DOC_RES_root)

  #File Menu Starts
  FileMenu = Menu(MenuBar, tearoff=0)

  #To Open already existing file
  FileMenu.add_command(label="Open", command = openFile(TextArea,value))

  MenuBar.add_cascade(label = "File", menu=FileMenu)
  # File Menu ends

  # Edit Menu Starts

  # Edit Menu Ends

  # Help Menu Ends

  DOC_RES_root.config(menu=MenuBar)

  #Adding Scrollbar using rules from Tkinter lecture no 22
  Scroll = Scrollbar(TextArea)
  Scroll.pack(side=RIGHT,  fill=Y)
  Scroll.config(command=TextArea.yview)
  TextArea.config(yscrollcommand=Scroll.set)
  return TextArea

#===========MainQueryFunction===============
def getvals():

    #QueryRELATEDPROCEDURE
        #Taking input and storing in list
        query=seachQuery.get()
        query = Convert(query)
        ##PreProcessing the Query 
        query = PreprocessingQuery(query)
        
        #reading from the disk 
        with open("VSM.json", 'r') as ii:
            global_VSM = json.load(ii)


        with open("UniqueList.json", 'r') as ij:
            uniquelist = json.load(ij)
        
        global_query_TF = calTermFrequency_query(query,uniquelist)
        print(query)
        save(global_query_TF ,"query")

    #COSINESIM_MAIN
        # now i have 2 global varible 
        # i) global_query_TF
        # ii) global_VSM
        COS_SIM = {}
        #storing the magnitude of query as it will use in all doc
        mag_query = magnitude(global_query_TF['query'])
        # print(global_VSM[str(1)])
        #calculating for each document 
        for i in range(50) :
            numerator = DotProduct(global_VSM[str(i+1)],global_query_TF['query'])
            mag_doc =  magnitude(global_VSM[str(i+1)])
            denominator = mag_doc * mag_query 
            ans = numerator / denominator
            COS_SIM.update({'SIMDOC{0}'.format(i+1):ans})



        #==========================CALCULATING THE COSINE SIM==============================
        sidebar_label = []
        j = 0
        for i in range(len(COS_SIM)) :
            val = COS_SIM['SIMDOC{0}'.format(i+1)]
            if val > 0.00498:
                sidebar_label.append('{0}'.format(i+1)) #storing the docNo in array 
                j += 1
                print(val)

        if j > 0 :        
            
            #===========================================
                #Making another window for result-Set
            #===========================================


            RESULT_win = Tk()
            
            #=========================================================================
          
            def checkNum(event,Listt,tt) :
                
                selection = event.widget.curselection()
                if selection:
                    index = selection[0]
                    data = event.widget.get(index)
                    
                    openFile(tt,data)

                else:
                    pass
                    

            #================================================================================

            scrollbar = Scrollbar(RESULT_win)
            scrollbar.pack( side = RIGHT, fill = Y )
            mylist = Listbox(RESULT_win, yscrollcommand = scrollbar.set ,width=20, fg='blue',
                                font=('Arial',16,'bold'))
            #====================================================================================
            total_columns =  2
            total_rows = j
            mylist.insert(END, 'length={0}'.format(j))
            
            #TABLE 
            for i in range(total_rows) :
                        
                m = Entry(RESULT_win, width=20, fg='blue',
                                font=('Arial',16,'bold'))
    
                mylist.insert(END, sidebar_label[i]) 
                #mylist.pack()
                mylist.pack( side = LEFT, fill = BOTH )
                scrollbar.config( command = mylist.yview )

            #------------------------------------------------------------------------------------------------------------
            #                                              READING THE DOCUMENT HEADING
            #------------------------------------------------------------------------------------------------------------
            
            #############################################
            #       Read Function
            def Read(listt,content) :
                    global file
                    
                    file = "E:/Semester_6/IR/Assignment#02/ShortStories/{0}.txt".format(content)
                    print(file)
                    if file == "":
                        file = None
                    else :
                        f = open(file, "r",encoding="utf-8")
                        listt.insert(END, 'DOC:{0}->{1}'.format(content,f.readline()) )
                        f.close()

            ###########################################
            scrollbar1 = Scrollbar(RESULT_win)
            scrollbar1.pack( side = LEFT, fill = Y )
            listt = Listbox(RESULT_win, yscrollcommand = scrollbar1.set ,width=40, fg='black',
                                font=('Arial',16))
            #====================================================================================
            total_columns =  2
            total_rows = j
            listt.insert(END, 'length={0}'.format(j))
            
            #TABLE 
            for i in range(total_rows) :
                        
                mm = Entry(RESULT_win, width=20, fg='blue',
                                font=('Arial',16,'bold'))
                Read(listt,sidebar_label[i])
                #listt.insert(END, "hello world") 
                #mylist.pack()
                listt.pack( side = LEFT, fill = BOTH )
                scrollbar.config( command = listt.yview )
            #-------------------------------------------------------------------------------------------------------------------

            #=========================Binding the function===================================
            # 
            # FUNCTION HIDEN
            DOC_RES_root = Tk()

            DOC_RES_root.title("RESULT DOCUMENT")
            #root.wm_iconbitmap("1.ico")
            DOC_RES_root.geometry("644x788")
            tt = Result_screen(2,DOC_RES_root)
            def handler(event): 
                
                
               
                a = checkNum(event, mylist,tt) 
                
                return a
           
            mylist.bind('<Button-1>', handler) 
            DOC_RES_root.mainloop()
           
              #----------------------------------------------------------------
          
        #===========
        # TextArea = Text(root, font="lucida 13")
        # file = None
        # TextArea.pack(expand=True, fill=BOTH)

#=======================================================================================

root.geometry("644x344")

# Tkinter variable for storing entries
seachQuery = StringVar()

#Entries for our form
nameentry = Entry(root, textvariable=seachQuery)

# Packing the Entries
nameentry.place(x=230,y=180)

#Button & packing it and assigning it a command
Button(text="Search", command=getvals).place(x=360,y=175)

root.mainloop()

yes i am here 
yes i am here 
['master', 'jacket']
0.005412815750213895
0.007772675088948306
0.028062002266522352
0.005191689293232048
0.009289797808332686
0.013818916584620936
0.007573767804185136
0.050897715044419575
E:/Semester_6/IR/Assignment#02/ShortStories/6.txt
E:/Semester_6/IR/Assignment#02/ShortStories/11.txt
E:/Semester_6/IR/Assignment#02/ShortStories/12.txt
E:/Semester_6/IR/Assignment#02/ShortStories/16.txt
E:/Semester_6/IR/Assignment#02/ShortStories/18.txt
E:/Semester_6/IR/Assignment#02/ShortStories/22.txt
E:/Semester_6/IR/Assignment#02/ShortStories/28.txt
E:/Semester_6/IR/Assignment#02/ShortStories/41.txt
Here i am  2
E:/Semester_6/IR/Assignment#02/ShortStories/2.txt
Here i am  11
E:/Semester_6/IR/Assignment#02/ShortStories/11.txt
